In [13]:
import matplotlib.pyplot as plt
import os
import re
import shutil
import string
import tensorflow as tf
import numpy as np
import requests
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup 
from keras import layers
from keras import losses

In [21]:
import pandas as pd

filepath_dict = {'trainingData' : '/Users/jacobjohnson/data_sets/congressional_tweet_training_data.csv', 'testData' : '/Users/jacobjohnson/data_sets/congressional_tweet_test_data.csv'}

df_list = []
for source, filepath in filepath_dict.items():
    df = pd.read_csv(filepath, names=['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year', 'party_id'], skipinitialspace=True, skiprows=1, sep=',')
    # df['source'] = source
    df_list.append(df)

df = pd.concat(df_list)
print(df.iloc[1])
print(df.columns)

favorite_count                                                  258
full_text         b"Today I'm urging the @CDCgov to immediately ...
hashtags                                                Coronavirus
retweet_count                                                   111
year                                                         2020.0
party_id                                                          R
Name: 1, dtype: object
Index(['favorite_count', 'full_text', 'hashtags', 'retweet_count', 'year',
       'party_id'],
      dtype='object')


In [15]:
import pandas as pd
import tensorflow_datasets as tfds
from sklearn.model_selection import train_test_split


df_training = df

hashtags = df_training['hashtags'].values
y = df_training['party_id'].values

# ds_split, info = tfds.load('trainingData', split=['train[:20%]', 'train[20%:]'], as_supervised=True, with_info=True)


hashtags_train, hashtags_test, y_train, y_test = train_test_split(hashtags, y, test_size=0.25, random_state=1000)



In [16]:
import pandas as pd
import numpy as np
import tensorflow as tf
from keras import layers

np.set_printoptions(precision=3, suppress=True)

training_data = np.array(df)


In [17]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()
vectorizer.fit(hashtags_train)

X_train = vectorizer.transform(hashtags_train)
X_test  = vectorizer.transform(hashtags_test)
X_train

<643352x79894 sparse matrix of type '<class 'numpy.int64'>'
	with 948308 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression(max_iter=1000)
classifier.fit(X_train, y_train)
score = classifier.score(X_test, y_test)

print("Accuracy: ", score)

Accuracy:  0.7679190118022299


In [19]:
from keras.models import Sequential
from keras import layers

input_dim = X_train.shape[1]

# model = Sequential()
# model.add(layers.Dense(10, input_dim=input_dim, activation='relu'))
# model.add(layers.Dense(1, activation='sigmoid'))


model = Sequential([
  tf.keras.layers.Dense(10, input_dim=input_dim, activation=tf.nn.relu),
  tf.keras.layers.Dense(10, activation=tf.nn.relu),
  tf.keras.layers.Dense(2)
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 10)                798950    
                                                                 
 dense_1 (Dense)             (None, 10)                110       
                                                                 
 dense_2 (Dense)             (None, 2)                 22        
                                                                 
Total params: 799,082
Trainable params: 799,082
Non-trainable params: 0
_________________________________________________________________


2022-05-04 13:18:16.818699: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [20]:
# input_tensor = tf.convert_to_tensor(training_data)
# # input_tensor = tf.constant(df)

# predictions = model(input_tensor)
# predictions[:5]
# tf.nn.softmax(predictions[:5])

train_features = training_data.copy()
train_labels = train_features['party_id']


training_data.fit(train_features, train_labels, epochs=10)



IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices